# TensorFlow Recommenders: Quickstart

To estimate a simple matrix factorization model on the Movielens 100K dataset, the following is enough:

1. Import TFRS

In [0]:
from typing import Dict, Text

import tensorflow as tf

import tensorflow_recommenders as tfrs

### Read the data

In [0]:
ratings, movies = tfrs.datasets.movielens_100K()

### Define a model:

In [0]:
class Model(tfrs.Model):

  def __init__(self):
    super().__init__()

    # Set up user representation.
    self.user_model = tf.keras.layers.Embedding(
        input_dim=2000, output_dim=64)
    # Set up movie representation.
    self.item_model = tf.keras.layers.Embedding(
        input_dim=2000, output_dim=64)
    # Set up a retrieval task and evaluation metrics over the
    # entire dataset of candidates.
    self.task = tfrs.tasks.RetrievalTask(
        corpus_metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(lambda x: self.item_model(x["movie_id"]))
        )
    )

  def train_loss(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.item_model(features["movie_id"])

    return self.task(user_embeddings, movie_embeddings)

### Fit and evaluate it.

In [4]:
model = Model()
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

# Train.
model.fit(train.batch(4096), epochs=5)

# Evaluate.
model.evaluate(test.batch(4096), return_dict=True)

Epoch 1/5
20/20 [==============================] - 2s 113ms/step - factorized_top_k: 0.0121 - factorized_top_k/top_1_categorical_accuracy: 5.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0015 - factorized_top_k/top_10_categorical_accuracy: 0.0049 - factorized_top_k/top_50_categorical_accuracy: 0.0421 - loss: 33378.4929
Epoch 2/5
20/20 [==============================] - 2s 113ms/step - factorized_top_k: 0.0311 - factorized_top_k/top_1_categorical_accuracy: 1.3750e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0049 - factorized_top_k/top_10_categorical_accuracy: 0.0142 - factorized_top_k/top_50_categorical_accuracy: 0.1053 - loss: 31086.8523
Epoch 3/5
20/20 [==============================] - 2s 115ms/step - factorized_top_k: 0.0487 - factorized_top_k/top_1_categorical_accuracy: 4.1250e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0093 - factorized_top_k/top_10_categorical_accuracy: 0.0258 - factorized_top_k/top_50_categorical_accuracy: 0.1592 - loss: 30333.6686
E

{'factorized_top_k': array([0.     , 0.00015, 0.0017 , 0.05525], dtype=float32),
 'factorized_top_k/top_1_categorical_accuracy': 0.0,
 'factorized_top_k/top_5_categorical_accuracy': 0.0001500000071246177,
 'factorized_top_k/top_10_categorical_accuracy': 0.0017000000225380063,
 'factorized_top_k/top_50_categorical_accuracy': 0.05525000020861626,
 'loss': 30111.48828125}